# Document retrieval

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import xml.etree.ElementTree as ET

import utils.manage_files

In [5]:
# Mount the drive shared folder to load pre-saved files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We suggest you to run this notebook on Google Colab because we used the Google Colab Forms to make the cells clearer.

***Instructions for running the notebook during tests***:
- From the **Download / Load datasets** section you need to import the downloads.tar.gz file, then run the cells to load the clueweb12 corpus and the topics.
- From **Indexes creation or import** section run all the cells where there are pip commands, the cell where the class DocumentsIndex is declared, the one for importing the zipped indexes.
- Then you can run the last section for tests.

## Download / Load datasets

Run the following cell in order to load the needed datasets from the Drive shared folder.

In [ ]:
!cp /content/drive/MyDrive/NLP_project/downloads.tar.gz .
!tar -xvf downloads.tar.gz
!rm downloads.tar.gz

### The ClueWeb12 corpus
We can dowload, or open it in case we load it from Drive, the corpus that we need to use for the task. If you load it from Drive you will find in the dataframe a column for the original text and one with the clean text.

In [3]:
url_corpus = "https://zenodo.org/record/6802592/files/touche-task2-passages-version-002.jsonl.gz?download=1"
zip_path_corpus = "corpus.jsonl.gz"
file_path_corpus = "corpus.jsonl"

download_corpus = utils.manage_files.DownloadFile(file_path_corpus, zip_path_corpus, url_corpus)
download_corpus()

'/content/downloads/corpus.jsonl' already present


In [4]:
corpus_df = pd.read_json(download_corpus.file_name, lines=True)
corpus_df.head()

,id,contents,clean,chatNoirUrl
0,clueweb12-0000tw-14-21168___1,"Shuga: Love, Sex, Money MTV Shuga Home Swag Bl...",shuga love sex money mtv shuga home swag blog ...,https://chatnoir.eu/cache?uuid=f338e91e-a3e9-5...
1,clueweb12-0000tw-14-21168___2,We LOVE sending #TeamShuga the exclusives. Ban...,love send teamshuga exclusive banky w wizkid l...,https://chatnoir.eu/cache?uuid=f338e91e-a3e9-5...
2,clueweb12-0000tw-14-21168___3,Now take note.. because you will be seeing a w...,note see lot risingstar month shuga set nairob...,https://chatnoir.eu/cache?uuid=f338e91e-a3e9-5...
3,clueweb12-0000tw-22-19226___1,Sex and love: The modern matchmakers | The Eco...,sex love modern matchmaker economist log regis...,https://chatnoir.eu/cache?uuid=2bf4b08d-2f65-5...
4,clueweb12-0000tw-22-19226___2,But have they? Feb 11th 2012 | from the print ...,feb print edition tweet long human romance wan...,https://chatnoir.eu/cache?uuid=2bf4b08d-2f65-5...


In [5]:
print(f"The corpus has {len(corpus_df)} elements.")

The corpus has 868655 elements.


### Topics, Quality and Relevance datasets

At this point we can download:
- the list of topics to retrieve all the titles;
- the relevance qrels file;
- the quality qrels file.

The last 2 files contain a relevance and quality scores, associated to a list of documents, with respect to a certain topic.

The structure of a .qrels file is:   
TOPIC 0 DOC_ID SCORE

We can use these data to train our document retrieval model on labeled data from past years.

#### Topics

In [6]:
# Download and parse the xml file of the 0-50 topics
url_topics = "https://zenodo.org/record/6873559/files/topics-task-2.zip?download=1"
zip_path_topics = "topics-task-2.zip"
file_path_topics = "topics-task-2"

download_topics = utils.manage_files.DownloadFile(file_path_topics, zip_path_topics, url_topics)
download_topics()

# First 50 topics
mytree = ET.parse(f"{download_topics.file_name}/topics-task-2.xml")
myroot = mytree.getroot()
topics = list()
for item in myroot:
    for x in item:        
        if x.tag == "title": # Specify the field, e.g., title
            topics.append(x.text.strip())

# Topics from 51 to 100
url_topics_21 = "https://zenodo.org/record/6873565/files/topics-task-2-2021.zip?download=1"
zip_path_topics_21 = "topics-task-2-2021.zip"
file_path_topics_21 = "topics-task-2-2021"

download_topics_21 = utils.manage_files.DownloadFile(file_path_topics_21, zip_path_topics_21, url_topics_21)
download_topics_21()

# Last 50 topics
mytree = ET.parse(f"{download_topics_21.file_name}/topics-task2-51-100.xml")
myroot = mytree.getroot()
for item in myroot:
    for x in item:        
        if x.tag == "title": # Specify the field, e.g., title
            topics.append(x.text.strip())

'/content/downloads/topics-task-2' already present
'/content/downloads/topics-task-2-2021' already present


In [7]:
# We have the 50 topics pre-selected from the team
print(f"There are {len(topics)} topics.\n{topics}")

There are 100 topics.
['What is the difference between sex and love?', 'Which is better, a laptop or a desktop?', 'Which is better, Canon or Nikon?', 'What are the best dish detergents?', 'What are the best cities to live in?', 'What is the longest river in the U.S.?', 'Which is healthiest: coffee, green tea or black tea and why?', 'What are the advantages and disadvantages of PHP over Python and vice versa?', 'Why is Linux better than Windows?', 'How to sleep better?', 'Should I buy an LCD TV or a plasma TV?', 'Train or plane? Which is the better choice?', 'What is the highest mountain on Earth?', 'Should one prefer Chinese medicine or Western medicine?', 'What are the best washing machine brands?', 'Should I buy or rent?', 'Do you prefer cats or dogs, and why?', 'What is the better way to grill outdoors: gas or charcoal?', 'Which is better, MAC or PC?', 'What is better: to use a brush or a sponge?', 'Which is better, Linux or Microsoft?', 'Which is better, Pepsi or Coke?', 'What is b

#### Documents relevance for each topic

In [61]:
# Download relevance qrels 50 topics
url_relevance = "https://zenodo.org/record/6873567/files/touche-task2-2022-relevance.qrels?download=1"
file_path_rel = "relevance.qrels"

download_relevance = utils.manage_files.DownloadFile(file_path_rel, url=url_relevance)
download_relevance()

'/content/downloads/relevance.qrels' already present


In [62]:
relevance_df = pd.read_csv(download_relevance.file_name, index_col=None, 
                    names=["topic", "0", "doc_id", "relevance"], sep=" ").drop("0", axis=1)

relevance_df.head()

,topic,doc_id,relevance
0,12,clueweb12-0002wb-18-34442___2,0
1,12,clueweb12-0004wb-69-30215___112,0
2,12,clueweb12-0004wb-78-20304___1,1
3,12,clueweb12-0004wb-78-20304___11,2
4,12,clueweb12-0008wb-62-05967___1,0


#### Documents quality for each topic

In [63]:
# Download relevance qrels 50 topics
url_quality = "https://zenodo.org/record/6873567/files/touche-task2-2022-quality.qrels?download=1"
file_path_qual = "quality.qrels"

download_quality = utils.manage_files.DownloadFile(file_path_qual, url=url_quality)
download_quality()

'/content/downloads/quality.qrels' already present


In [64]:
quality_df = pd.read_csv(download_quality.file_name, index_col=None, 
                    names=["topic", "0", "doc_id", "quality"], sep=" ").drop("0", axis=1)

quality_df.head()

,topic,doc_id,quality
0,12,clueweb12-0002wb-18-34442___2,2
1,12,clueweb12-0004wb-69-30215___112,2
2,12,clueweb12-0004wb-78-20304___1,2
3,12,clueweb12-0004wb-78-20304___11,2
4,12,clueweb12-0008wb-62-05967___1,0


#### Merge data

Now we want to merge the data in order to have the relevance and quality score in the same dataframe.

In [ ]:
det_df = relevance_df.merge(quality_df, on=['doc_id', 'topic'])
det_df.head()

,topic,doc_id,relevance,quality
0,12,clueweb12-0002wb-18-34442___2,0,2
1,12,clueweb12-0004wb-69-30215___112,0,2
2,12,clueweb12-0004wb-78-20304___1,1,2
3,12,clueweb12-0004wb-78-20304___11,2,2
4,12,clueweb12-0008wb-62-05967___1,0,0


In [ ]:
def retrieve_by_topic_relevance(topic, relevance, det_df, corp_df):
    print(f"Topic: {topic} - Relevance: {relevance}")
    id_list = det_df[(det_df['topic']==topic) & (det_df['relevance']==relevance)]['doc_id']
    url_list = []
    for doc in id_list:
        if doc in corp_df.id.values:
            url_list.append(corp_df[corp_df['id']==doc]['chatNoirUrl'].item()) 
    return url_list

In [ ]:
# List of topics taken from qrels files.
np.sort(det_df.topic.unique())

array([  2,   3,   8,   9,  12,  14,  17,  18,  19,  22,  23,  25,  26,
        27,  28,  30,  33,  34,  36,  37,  42,  43,  48,  51,  53,  54,
        55,  56,  58,  59,  60,  61,  62,  67,  68,  69,  70,  72,  74,
        76,  77,  78,  84,  86,  88,  91,  92,  93,  95, 100])

In this example we retrieve the documents that discuss about chiropractic therapy or physical one and they are perfectly related to this argument.

In [ ]:
example_topic = 17
retrieve_by_topic_relevance(example_topic, 2, det_df, corpus_df)

In [ ]:
topics[example_topic-1]

'Do you prefer cats or dogs, and why?'

## Documents pre-processing

It's important to perform some pre-processing in order to capture better the similarities between documents and removing the noise from them. Obviously if you downloaded the 'downloads' directory from Drive you don't have to perform the preprocessing and you will already find the clean text.

In [ ]:
!pip install contractions -q 
!pip install spacy -q

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.1/110.1 KB 10.7 MB/s eta 0:00:00


In [ ]:
import contractions
import spacy

import string
# To enable progress bar in apply function
from tqdm.notebook import tqdm
tqdm.pandas()

nlp = spacy.load("en_core_web_sm", disable=['ner','parser'])
nlp.max_length=5000000

### Clean text

In the following cells we decided to clean the documents in this way:
1. Make the documents lowercase.
2. Expand contractions.
3. Remove words with numbers inside.
4. Replace \n, characters that are not in the english alphabet and punctuation with a space.
5. Remove adjacent spaces.
6. Remove URLs and stopwords.
7. Perform lemmatization.

In [ ]:
# Clean the documents performing pre-processing
def clean_documents(text, nlp):
    clean = text.lower()
    clean = contractions.fix(clean)
    clean = clean.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    clean = re.sub('\w*\d\w*','', clean)
    clean = re.sub('\n',' ', clean)
    clean = re.sub(r"https?:/*\S+", "", clean)
    clean = re.sub('[^a-z]',' ', clean)
    clean = re.sub(' +',' ', clean)

    clean = ' '.join([token.lemma_ for token in list(nlp(clean)) if not token.is_stop])
    return clean

In [ ]:
corpus_df['contents'] = corpus_df['contents'].progress_apply(clean_documents, nlp=nlp)

  0%|          | 0/868655 [00:00<?, ?it/s]

  0%|          | 0/868655 [00:00<?, ?it/s]

  0%|          | 0/868655 [00:00<?, ?it/s]

## Indexes creation or import

Given that we are not able to get an API key for the ChatNoir system, we have to create our own index for ranking the documents with respect to a certain query.

We decided to use the pyserini library, but it doesn't support the creation of a dense index for a custom dataset, therefore we used also the autofaiss library. 

In [8]:
!pip install -q pyserini
!pip install -q faiss-cpu==1.7.2
!pip install -q autofaiss

In [9]:
#@title ###The class for a sparse or dense index
#@markdown In this cell there is the class to represent a dense or sparse index
#@markdown and the relative function to make a search on it given a query.
import shutil

import faiss
from pyserini.search import LuceneSearcher, TctColBertQueryEncoder

class DocumentsIndex:
    allowed_zip_format = ['zip', 'tar', 'tar.gz']

    def __init__(self, index_path: str, index_type: str) -> None:
        if self.__path_analysis(index_path):
            self.index_path = index_path
            self.index_type = index_type if index_type in ['dense', 'sparse'] else 'sparse'
            self.index = None

            # Only for the dense index
            if self.index_type == 'dense':
                self.enc_name = "castorini/tct_colbert-v2-hnp-msmarco"
                self.encoder = None

        self.__load_index()


    def __path_analysis(self, path: str):
        '''
            It checks if the file exists and if it is a zip it unpacks it.
            It returns False if an error occurs, otherwise True.
        '''
        if not os.path.exists(path):
            print(f"ERROR: the index doesn't exist at the following path: {path}")
            return False
        else:
            _, ext = os.path.splitext(path)
            # Unpack the file
            if ext in self.allowed_zip_format:
                print("WARNING: the input path is of a compressed file, now it will be unpacked.")
                shutil.unpack_archive(path)

        return True


    def __load_index(self):
        '''
            It actually loads the index into a variable.
        '''
        print(f"Loading the {self.index_type} index file ...")
        if self.index_type == "sparse":
            self.index = LuceneSearcher(self.index_path)
        else:
            self.index = faiss.read_index(self.index_path)
            print(f"Loading the encoder {self.enc_name} ...")
            self.encoder = TctColBertQueryEncoder(self.enc_name)

        print("\nThe process is finished correctly!\n")
                
    
    def __index_to_docid(self, indices, corpus_df):
        '''
            It converts indices to document ids (for dense index).
        '''
        indices = indices.tolist()
        for ind in range(len(indices)):
            indices[ind] = corpus_df.iloc[indices[ind]]['id'].tolist()
        return indices
    

    def search(self, query: str, k: int=10, corpus_df=None, verbose=False):
        ''' 
            If the query is a single string it returns a list of tuples (id, score),
            otherwise it returns a dictionary with keys the numbers with the position
            of the query in the array. For each query you will find the same array
            as for the single query. 
            E.g.
                query = "Dogs or cats?"
                res = [(doc_id1, score1), (doc_id2, score2), ...]

                query = ["Dogs or cats?", "Coke or Pepsi?"]
                res = {'0':[(doc_id1, score1),...], '1': [(doc_id1, score1,...)]}
        '''
        if self.index_type == 'sparse':
            if isinstance(query, list):
                hits = self.index.batch_search(query, [str(i) for i in range(len(query))], k=k)
                for key in hits.keys():
                    hits[key] = [(el.docid, el.score) for el in hits[key]]
            else:
                hits = self.index.search(query, k=k)
                hits = [(el.docid, el.score) for el in hits]
        else:
            if corpus_df is None:
                print("ERROR: corpus needed for the dense index in order to convert the indices to the document ids.")
                return None
            if isinstance(query, list):
                embeddings = []
                # Create the embedding for each query and stack them to create a batch
                for q in query:
                    query_enc = self.encoder.encode(q)
                    query_enc = np.expand_dims(query_enc, axis=0)
                    embeddings.append(query_enc)

                distances, indices = self.index.search(np.concatenate(embeddings), k)
                # Convert indices to document ids
                indices = self.__index_to_docid(indices, corpus_df)

                hits = {str(i): list(zip(indices[i], distances[i])) 
                            for i in range(len(query))}
            else:
                query_enc = self.encoder.encode(query)
                query_enc = np.expand_dims(query_enc, axis=0)
                distances, indices = self.index.search(query_enc, k)
                # Convert indices to document ids
                indices = self.__index_to_docid(indices, corpus_df)
                hits = list(zip(indices[0], distances[0]))

        if verbose:
            self.__print_results(hits)
        return hits


    def __print_results(self, results):
        '''
            It prints the results of the search given the id and the score ordered
            considering the score.
        '''
        if isinstance(results, dict):
            for key in results.keys():
                print(f"{int(key)+1}. For the query {key}:")
                for i, (ind, score) in enumerate(results[key]):
                    print(f'{i+1:2} {ind:4} {score:.4f}')
        else:
            for i, (ind, score) in enumerate(results):
                    print(f'{i+1:2} {ind:4} {score:.4f}')

In [ ]:
#@title #### Create a corpus without chatNoirUrl for sparse index (execute only if you don't have pre-saved indexes)
#@markdown In this cell you save a jsonl file without the 'chatNoirUrl' attribute that is not required for creating the index.
# Save the jsonl file for creating the index
!mkdir collections
corpus_df.drop('chatNoirUrl', axis=1).to_json('collections/corpus_index.jsonl', orient="records", lines=True)

Load the pre-saved indexes from the Drive folder. For the sparse index you will find a directory 'sparse_index', for the dense one you will find a file named 'knn.index'.

In [ ]:
# Load the indexes from Drive
!cp /content/drive/MyDrive/NLP_project/dirty_indexes.zip .
!unzip dirty_indexes.zip
!rm dirty_indexes.zip

In [21]:
# Retrieve the urls from the corpus given the results of the search on the index
def retrieve_docs(corpus, hits, mode="sparse"):
    urls = []
    if mode=='sparse':
        for el in hits:
            urls.append(corpus[corpus['id'] == el[0]]['chatNoirUrl'].item())
        ids = [val[0] for val in hits]
    else:
        indices = [ind[0] for ind in hits]
        urls = corpus.iloc[indices]['chatNoirUrl'].values
        ids = corpus.iloc[indices]['id'].values
    return ids, urls

### Sparse index Test

The sparse index uses the BM25 score to rank the documents with respect to a given query.

The following cell show the command that we executed to create the index the first time.

In [ ]:
#@title #### Create the sparse index
#@markdown Execute this cell to create a sparse index if not pre-saved, the .jsonl corpus in this case needs to be inside 'collections' directory. 
!python -m pyserini.index.lucene --collection JsonCollection --input collections/ --index indexes/sparse_index --bm25.accurate --generator DefaultLuceneDocumentGenerator --threads 2 --storePositions --storeDocvectors --storeRaw

In order to retrieve the documents from the index we can use the LuceneSearcher from pyserini and ask for the top-k documents, considering the BM25 w.r.t. a query.

In [ ]:
#@title Test with rm3 query expansion, cell to remove in the future
'''
TODO:
    - if we want to use it we need to understand hyperparameters
    - add it to the class because we need to set it to the searcher.
    - WARNING: get feedback terms doesn't work if rm3 was not set 
''';

#fb_terms (int) – number of terms to add to the query. Terrier’s default setting is 10 expansion terms.
#fb_docs (int) – number of feedback documents to consider. Terrier’s default setting is 3 feedback documents.
#fb_lambda (float) – lambda in RM3, i.e. importance of relevance model viz feedback model. Defaults to 0.6.

a = LuceneSearcher('indexes/sparse_index')
a.set_rm3(10, 10, 0.5)

sorted(a.get_feedback_terms("Coke or Pepsi?").items(), key=lambda x: x[1], reverse=True)

In [ ]:
sparse_index = DocumentsIndex("indexes/sparse_index", "sparse")

Loading the sparse index file ...

The process is finished correctly!



In [ ]:
sparse_query = ["Coke or Pepsi?", "Dogs or cats?"]
sparse_results = sparse_index.search(sparse_query, k=10, verbose=True)

1. For the query 0:
 1 clueweb12-1301wb-59-03888___1 11.5156
 2 clueweb12-1412wb-02-24400___2 11.2515
 3 clueweb12-1802wb-28-00297___4 11.1894
 4 clueweb12-0304wb-70-29640___3 11.1401
 5 clueweb12-1000tw-98-11983___1 11.1026
 6 clueweb12-1800tw-33-09364___3 11.0931
 7 clueweb12-0406wb-48-23319___2 11.0624
 8 clueweb12-1308wb-96-22768___3 11.0054
 9 clueweb12-1108wb-45-26706___72 11.0006
10 clueweb12-0800tw-75-14494___6 10.9743
2. For the query 1:
 1 clueweb12-0002wb-08-13311___5 8.2143
 2 clueweb12-1014wb-63-07431___2 8.1670
 3 clueweb12-1609wb-04-14607___1 8.1578
 4 clueweb12-0210wb-51-10734___1 8.1387
 5 clueweb12-1608wb-75-26810___1 8.1239
 6 clueweb12-0807wb-34-16439___1 8.1200
 7 clueweb12-0103wb-01-08587___30 8.1057
 8 clueweb12-0307wb-57-02563___13 8.0899
 9 clueweb12-0901wb-43-01355___4 8.0816
10 clueweb12-0912wb-43-13039___1 8.0763


In [ ]:
docids, urls = retrieve_docs(corpus_df, sparse_results['0'], "sparse")
urls

['https://chatnoir.eu/cache?uuid=822acef3-0d2e-58cb-a9a9-62c6a3e7b9a0&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=86a38a3b-58c0-530b-a660-56e2b4bffe2a&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=4ed41510-6cca-5c6b-a35c-d00d267a7c61&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=049caace-9ff8-5c8d-aa32-30b10e9a5889&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=51b9cf13-71b3-51c1-9847-fbddd4bc53cd&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=ce4a164e-d16b-5eb4-9316-2f0bab2b67b1&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=aacde075-d746-5462-8ca9-7a64b7135d3b&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=c4bf5091-a6db-5e5d-9ed6-43328bd6451c&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=d90b1021-15fc-5306-8804-37847ff52a08&index=cw12&raw&plain',
 'https://chatnoir.eu/cache?uuid=d243c30b-1dd6-5f8a-bc42-501eea6a2545&index=cw12&raw&plain']

### Dense index Test

In [ ]:
!pip install -q autofaiss
!pip install -q tqdm

First of all we need to compute an embedding vector for the content of each document, then we compute the knn index considering the inner product as score.

#### Compute the embeddings and create the index

You can compute the embeddings for each document considering a pre-trained version of ColBERT on MSMARCO, or you can load it from a jsonl file that we saved after the first computation.

In [ ]:
# Compute the embedding
!python -m pyserini.encode input --corpus collections/corpus_index.jsonl --fields text --shard-id 0 --shard-num 1 output --embeddings embeddings/ encoder --encoder castorini/tct_colbert-v2-hnp-msmarco --fields text --batch 32 --fp16

Downloading: 100% 559/559 [00:00<00:00, 988kB/s]
Downloading: 100% 438M/438M [00:06<00:00, 70.0MB/s]
Downloading: 100% 334/334 [00:00<00:00, 473kB/s]
Downloading: 100% 232k/232k [00:00<00:00, 29.2MB/s]
Downloading: 100% 112/112 [00:00<00:00, 200kB/s]
868655it [00:05, 153128.27it/s]
100% 27146/27146 [56:53<00:00,  7.95it/s]


In [ ]:
# Load the embeddings of the documents
!cp /content/drive/MyDrive/NLP_project/embeddings.jsonl.gz .
!gzip -d embeddings.jsonl.gz 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


After the loading of the file with the documents embedding we should transform the vectors to npy files in order to create the faiss index with autofaiss library. To avoid running out of RAM we decided to read the big file with the embeddings in chunks and then we saved different numpy files that we will use to create the index. 

In [ ]:
import tqdm

def convert_to_npy(path, file_len, chunksize=75000):
    '''
        It takes as input a .jsonl file and it creates some .npy files taking
        only the 'vector' key, that is the embedding of the documents.
    '''
    steps = file_len//chunksize
    for i, chunk in enumerate(tqdm.tqdm(pd.read_json(path, lines=True, chunksize=chunksize), total=steps)):
        npy_list = []
        for vect in chunk['vector'].to_numpy():
            npy_list.append(vect)

        # Save different files to avoid RAM consumption
        np.save(f'embeddings/embeddings_{i+10}.npy', np.array(npy_list))
        del npy_list

In [ ]:
file_len = 868655
convert_to_npy('embeddings.jsonl', file_len)

12it [04:27, 22.31s/it]


Given the .npy files with the embeddings autofaiss automatically generates a dense index for us (~ 1 hour).

In [ ]:
from autofaiss import build_index

# Load the .npy files from the "embeddings" directory where we saved them
build_index(embeddings="embeddings", index_path="indexes/knn.index",
            index_infos_path="indexes/dense_index_infos.json", max_index_memory_usage="6GB",
            current_memory_available="9GB")

Once we computed the dense index we saved on Drive such that we can load it and execute our search.

#### Test the dense index

To compute the embedding of the vectors I used '*castorini/tct_colbert-v2-hnp-msmarco*' pre-trained model thus we use it also for encoding the queries that we need to look for.

In [ ]:
dense_index = DocumentsIndex('indexes/knn.index', 'dense')

In [ ]:
dense_query = ["Coke or Pepsi?", "Dogs or cats?"]
#dense_query = clean_documents(dense_query, nlp)
dense_results = dense_index.search(dense_query, k=10, verbose=True)

1. For the query 0:
 1 163681 78.5340
 2 163014 78.4952
 3 163013 78.4652
 4 161391 78.4319
 5 163728 78.3653
 6 161807 78.3526
 7 163194 78.3127
 8 163377 78.2972
 9 160718 78.2814
10 164633 78.2748
2. For the query 1:
 1 130699 78.0054
 2 721741 77.9877
 3 128393 77.8762
 4 131011 77.8614
 5 131088 77.8470
 6 129754 77.8463
 7 132408 77.8407
 8 128390 77.7946
 9 128701 77.7920
10 735652 77.7899


In [ ]:
retrieve_docs(corpus_df, dense_results['1'], "dense")

(array(['clueweb12-0901wb-43-01355___4', 'clueweb12-1802wb-25-12690___12',
        'clueweb12-0110wb-12-26753___5', 'clueweb12-0907wb-66-32167___9',
        'clueweb12-0910wb-05-27531___1', 'clueweb12-0607wb-74-11201___8',
        'clueweb12-1212wb-04-20969___4', 'clueweb12-0110wb-12-26753___2',
        'clueweb12-0207wb-47-07548___3', 'clueweb12-0300wb-31-18303___54'],
       dtype=object),
 array(['https://chatnoir.eu/cache?uuid=b509c2e2-9474-565e-8a15-48ad94ce965c&index=cw12&raw&plain',
        'https://chatnoir.eu/cache?uuid=f8caa24f-8267-51f7-8cdf-ce32a24174e6&index=cw12&raw&plain',
        'https://chatnoir.eu/cache?uuid=34c87bfe-f1f6-5935-92eb-d130efdcdad4&index=cw12&raw&plain',
        'https://chatnoir.eu/cache?uuid=adfccc41-6db5-5084-ba5f-3b8d0673957f&index=cw12&raw&plain',
        'https://chatnoir.eu/cache?uuid=0c07141c-8d44-502e-ba6b-12aa2ba9a3c2&index=cw12&raw&plain',
        'https://chatnoir.eu/cache?uuid=366d8c51-7bf2-5992-8cd3-22e5f50d5f1d&index=cw12&raw&plain',
     

## Models

**TODO**
- make the compute_results function return the list of best documents or urls, to check manually the text passages.

In [34]:
def print_ndcg(ndcg_scores):
    if ndcg_scores and isinstance(ndcg_scores, dict):
        for key, ndcg in ndcg_scores.items():
            print(f"The nDCG for {key} qrel is:")
            print(ndcg, '\n')
    else:
        print("nDCG has not been computed, set 'evaluate=True' to compute it.")


# Retrieve the urls from the corpus given the results of the search on the index
def retrieve_docs_ranked(corpus, hits, k=10):
    urls = []
    for el in hits['ids'][:k]:
        urls.append(corpus[corpus['id'] == el]['chatNoirUrl'].item())
    ids = [val for val in hits['ids'][:k]]
    return ids, urls

In [ ]:
# Create the directory for the results
!mkdir results

In [11]:
sparse_index = DocumentsIndex('indexes/sparse_index', 'sparse')

Loading the sparse index file ...

The process is finished correctly!



In [12]:
dense_index = DocumentsIndex('indexes/knn.index', 'dense')

Loading the dense index file ...
Loading the encoder castorini/tct_colbert-v2-hnp-msmarco ...

The process is finished correctly!



### Dense Simple with dirty index

In [13]:
from src.DensePipeline import DensePipeline

dense_pipeline = DensePipeline('results', 'dense_pipeline', dense_index)

In [16]:
dense_scores, ndcg_dense = dense_pipeline.compute_results(
                                ['downloads/relevance.qrels', 'downloads/quality.qrels'],
                                topics, corpus_df, k=40, clean_query=True,
                                evaluate=True
                            )

print_ndcg(ndcg_dense)

The nDCG for relevance qrel is:
              Tag    nDCG@5
0  dense_pipeline  0.594256 

The nDCG for quality qrel is:
              Tag    nDCG@5
0  dense_pipeline  0.614225 



You can use the retrieved scores to make some further checks, for example reading the documents retrieved by the pipeline for a specific topic.

In [37]:
test_topic = 2
query = topics[test_topic-1]

_, test_urls = retrieve_docs_ranked(corpus_df, dense_scores[2], k=5)
print(f"Answer:\n{query}\n")
print("The top-k retrieved documents are:\n", '\n'.join(test_urls))

Answer:
Which is better, a laptop or a desktop?

The top-k retrieved documents are:
 https://chatnoir.eu/cache?uuid=4d97d1c9-37de-5595-9fe2-27b59f47ace7&index=cw12&raw&plain
https://chatnoir.eu/cache?uuid=8d498f77-74b7-54e7-ab83-6c8d50e5ab18&index=cw12&raw&plain
https://chatnoir.eu/cache?uuid=591619da-3135-597e-80d8-debf33f5daba&index=cw12&raw&plain
https://chatnoir.eu/cache?uuid=02b59616-116d-53ab-8414-885ef822bc3e&index=cw12&raw&plain
https://chatnoir.eu/cache?uuid=e528f057-6644-5814-9b3d-0b7d6b67e5fe&index=cw12&raw&plain


### Sparse simple with dirty index

In [38]:
from src.SparsePipeline import SparsePipeline

sparse_pipeline = SparsePipeline('results', "sparse_pipeline", sparse_index)

In [41]:
sparse_scores, ndcg_sparse = sparse_pipeline.compute_results(
                                ['downloads/relevance.qrels', 'downloads/quality.qrels'],
                                topics, k=40, evaluate=True,
                                clean_query=False
                             )

print_ndcg(ndcg_sparse)

The nDCG for relevance qrel is:
               Tag   nDCG@5
0  sparse_pipeline  0.46871 

The nDCG for quality qrel is:
               Tag    nDCG@5
0  sparse_pipeline  0.476255 



### Hybrid approach

In [42]:
from src.HybridPipeline import HybridPipeline

hybrid_pipeline = HybridPipeline('results', 'hybrid_pipeline', sparse_index, dense_index)

In [43]:
scores_hybrid, ndcg_hybrid = hybrid_pipeline.compute_results(
                                ['downloads/relevance.qrels', 'downloads/quality.qrels'],
                                topics, corpus_df, k=700, alpha=0.2, evaluate=True
                             )

print_ndcg(ndcg_hybrid)

The nDCG for relevance qrel is:
               Tag    nDCG@5
0  hybrid_pipeline  0.613824 

The nDCG for quality qrel is:
               Tag    nDCG@5
0  hybrid_pipeline  0.646737 

